In [50]:
from rekall import IntervalSetMapping, IntervalSet, Interval
from rekall.bounds import Bounds1D
from rekall.predicates import *

In [3]:
import numpy as np

In [4]:
a = np.load('/dfs/scratch0/paroma/heart-MRI/bav-classification/coral/gen_unknown_L_dev.npy')

In [5]:
b = np.load('/dfs/scratch0/paroma/heart-MRI/bav-classification/coral/gen_unknown_y_dev.npy')

In [38]:
a.T[6:12]

array([[ 0.,  0.,  0.,  0., -1.],
       [ 0.,  1.,  0.,  0., -1.],
       [ 1.,  1.,  1.,  0., -1.],
       [ 1.,  1.,  1.,  0.,  1.],
       [ 0.,  1.,  0., -1.,  1.],
       [ 1.,  1.,  1., -1.,  1.]])

In [9]:
b.shape[0]

540

In [16]:
a.T.shape

(540, 5)

In [40]:
gt_ism_frame = IntervalSetMapping({
    i: IntervalSet([
        Interval(Bounds1D(frame, frame+1), payload = b.tolist()[i*6 + frame])
        for frame in range(6)
    ])
    for i in range(int(b.shape[0] / 6))
})

In [41]:
gt_ism_patient = gt_ism_frame.coalesce(
    ('t1', 't2'), Bounds1D.span
)

In [43]:
lf_ism = IntervalSetMapping({
    i: IntervalSet([
        Interval(Bounds1D(frame, frame+1), payload=lfs)
        for frame, lfs in enumerate(a.T.tolist()[i * 6:i*6 + 6])
    ])
    for i in range(int(a.T.shape[0] / 6))
})

In [44]:
frame_mv = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [1 if intrvl['payload'].count(1) > intrvl['payload'].count(-1) else -1]
    )
)

In [48]:
patient_mv = frame_mv.coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = 1 if intrvl['payload'].count(1) > intrvl['payload'].count(-1) else -1
    )
)

In [52]:
len(patient_mv.filter(payload_satisfies(lambda p: p == 1)))

33

In [96]:
def tp_fp_fn_patient(predicted, gt):
    tp = predicted.filter(
        payload_satisfies(lambda p: p == 1)
    ).filter_against(
        gt,
        predicate = lambda i1, i2: i1['payload'] == i2['payload']
    )
    
    fp = predicted.filter(
        payload_satisfies(lambda p: p == 1)
    ).minus(tp)
    
    fn = gt.filter(
        payload_satisfies(lambda p: p == 1)
    ).minus(
        predicted.filter(
            payload_satisfies(lambda p: p == 1)
        )
    )
    
    return tp, fp, fn

In [53]:
def prf1_patient(predicted, gt):
    tp = predicted.filter(
        payload_satisfies(lambda p: p == 1)
    ).filter_against(
        gt,
        predicate = lambda i1, i2: i1['payload'] == i2['payload']
    )
    
    fp = predicted.filter(
        payload_satisfies(lambda p: p == 1)
    ).minus(tp)
    
    fn = gt.filter(
        payload_satisfies(lambda p: p == 1)
    ).minus(
        predicted.filter(
            payload_satisfies(lambda p: p == 1)
        )
    )
    
    pre = len(tp) / (len(tp) + len(fp))
    rec = len(tp) / (len(tp) + len(fn))
    f1 = 2 * pre * rec / (pre + rec)
    
    return pre, rec, f1, len(tp), len(fp), len(fn)

In [54]:
prf1_patient(patient_mv, gt_ism_patient)

(0.15151515151515152, 0.7142857142857143, 0.25, 5, 28, 2)

In [75]:
threshold_frame = 0
threshold_seq = 0

In [76]:
attempt2 = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [
            1 if intrvl['payload'].count(1) > intrvl['payload'].count(-1) + threshold_frame
            else -1]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            1 if intrvl['payload'].count(1) > intrvl['payload'].count(-1) + threshold_seq
            else -1
        )
    )
)

In [77]:
prf1_patient(attempt2, gt_ism_patient)

(0.15151515151515152, 0.7142857142857143, 0.25, 5, 28, 2)

In [94]:
second_lf_unsure = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][1]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            1 if intrvl['payload'].count(1) > 4
            else -1
        )
    )
)

In [95]:
prf1_patient(second_lf_unsure, gt_ism_patient)

(0.24, 0.8571428571428571, 0.375, 6, 19, 1)

In [114]:
fourth_lf_negative = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][3]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            -1 if intrvl['payload'].count(-1) in (2, 3)
            else 1
        )
    )
)

In [115]:
prf1_patient(fourth_lf_negative, gt_ism_patient)

(0.09859154929577464, 1.0, 0.17948717948717946, 7, 64, 0)

In [117]:
second_fourth = second_lf_unsure.join(
    fourth_lf_negative,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
)

In [123]:
prf1_patient(
    second_fourth,
    gt_ism_patient
)

(0.3, 0.8571428571428571, 0.4444444444444444, 6, 14, 1)

In [118]:
tp, fp, fn = tp_fp_fn_patient(second_fourth, gt_ism_patient)

In [127]:
second_has_zero = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][1]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            -1 if intrvl['payload'].count(0) > 0
            else 1
        )
    )
)

In [128]:
prf1_patient(second_has_zero, gt_ism_patient)

(0.1794871794871795, 1.0, 0.30434782608695654, 7, 32, 0)

In [129]:
attempt3 = second_fourth.join(
    second_has_zero,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
)

In [130]:
prf1_patient(attempt3, gt_ism_patient)

(0.375, 0.8571428571428571, 0.5217391304347825, 6, 10, 1)

In [135]:
fifth_lf_starts_or_ends_with_0 = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][4]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            -1 if intrvl['payload'][0] == 0 or intrvl['payload'][-1] == 0
            else 1
        )
    )
)

In [136]:
prf1_patient(fifth_lf_starts_or_ends_with_0, gt_ism_patient)

(0.09210526315789473, 1.0, 0.1686746987951807, 7, 69, 0)

In [138]:
attempt4 = attempt3.join(
    fifth_lf_starts_or_ends_with_0,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
)

In [140]:
prf1_patient(attempt4, gt_ism_patient)

(0.46153846153846156, 0.8571428571428571, 0.6, 6, 7, 1)

In [141]:
tp, fp, fn = tp_fp_fn_patient(attempt4, gt_ism_patient)

In [152]:
three_negatives_twice = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [-1 if intrvl['payload'].count(-1) >= 3 else 1]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            -1 if intrvl['payload'].count(-1) >= 2
            else 1
        )
    )
)

In [153]:
prf1_patient(three_negatives_twice, gt_ism_patient)

(0.08860759493670886, 1.0, 0.1627906976744186, 7, 72, 0)

In [166]:
fifth_lf_less_than_three_positive = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][4]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            -1 if (intrvl['payload'].count(-1) > 3 or
                  intrvl['payload'].count(-1) > 2 and
                  intrvl['payload'].count(0) == 1)
            else 1
        )
    )
)

In [167]:
prf1_patient(fifth_lf_less_than_three_positive, gt_ism_patient)

(0.10714285714285714, 0.8571428571428571, 0.19047619047619047, 6, 50, 1)

In [168]:
attempt5 = attempt4.join(
    three_negatives_twice,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
).join(
    fifth_lf_less_than_three_positive,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
)

In [169]:
prf1_patient(attempt5, gt_ism_patient)

(0.6, 0.8571428571428571, 0.7058823529411764, 6, 4, 1)

In [170]:
tp, fp, fn = tp_fp_fn_patient(attempt5, gt_ism_patient)

In [171]:
for k in fp:
    print_lfs(lf_ism[k])

[[ 0. -1.  0.  0. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]]
[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]]


In [172]:
for k in tp:
    print_lfs(lf_ism[k])

[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0. -1.  1.]
 [-1.  1.  0.  0.  1.]
 [-1.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1. -1.  1.]
 [ 1.  1.  0. -1.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[ 1.  1.  1.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]


In [142]:
for k in fp:
    print_lfs(lf_ism[k])

[[-1. -1.  0.  0. -1.]
 [-1.  1.  0. -1. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]
[[ 0. -1.  0.  0. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]]
[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  1.  0. -1.]
 [ 1.  1.  1.  0. -1.]]
[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0.  1.  0.  1.  0.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]


In [143]:
for k in tp:
    print_lfs(lf_ism[k])

[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0. -1.  1.]
 [-1.  1.  0.  0.  1.]
 [-1.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1. -1.  1.]
 [ 1.  1.  0. -1.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[ 1.  1.  1.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]


In [131]:
tp, fp, fn = tp_fp_fn_patient(attempt3, gt_ism_patient)

In [132]:
for k in fp:
    print_lfs(lf_ism[k])

[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1. -1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0.  1.  0.  1.  0.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1. -1.  0.  0. -1.]
 [-1.  1.  0. -1. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]
[[ 0. -1.  0.  0. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]]
[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  1.  0. -1.]
 [ 1.  1.  1.  0. -1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0

In [134]:
for k in tp:
    print_lfs(lf_ism[k])

[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0. -1.  1.]
 [-1.  1.  0.  0.  1.]
 [-1.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1. -1.  1.]
 [ 1.  1.  0. -1.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[ 1.  1.  1.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]


In [119]:
for k in fp:
    print_lfs(lf_ism[k])

[[ 1.  1.  1. -1. -1.]
 [ 0.  0.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]
[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  0.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1. -1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0. -1.  1.]
 [-1.  1.  0. -1.  1.]
 [ 0.  0.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0.  1.  0.  1.  0.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1. -1.  0.  0. -1.]
 [-1

In [120]:
tp, fp, fn = tp_fp_fn_patient(second_lf_unsure, gt_ism_patient)

In [108]:
sorted(list(fp.get_grouped_intervals().keys()))

[0, 1, 2, 18, 28, 29, 35, 39, 40, 45, 46, 52, 54, 56, 61, 76, 77, 80, 89]

In [110]:
sorted(list(fourth_lf_negative.filter(
    payload_satisfies(lambda p: p == 1)
).get_grouped_intervals().keys()))

[0,
 1,
 2,
 4,
 6,
 7,
 12,
 13,
 18,
 20,
 27,
 28,
 29,
 31,
 32,
 35,
 36,
 39,
 40,
 41,
 42,
 43,
 45,
 46,
 49,
 50,
 52,
 53,
 54,
 56,
 61,
 69,
 70,
 73,
 74,
 75,
 76,
 77,
 80,
 89]

In [98]:
for k in fp:
    print_lfs(lf_ism[k])

[[-1.  1.  0.  0.  1.]
 [-1.  0.  0.  0. -1.]
 [-1.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[ 0.  0.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 1.  1.  1.  0. -1.]
 [ 1.  1.  1.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1. -1.  1.]]
[[ 1.  1.  1. -1. -1.]
 [ 0.  0.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]
[[ 0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  0.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1. -1.  0.  0.  0.]
 [ 0

In [78]:
gt_ism_patient.filter(payload_satisfies(lambda p: p == 1))

{4: [<Interval t1:0 t2:6 payload:1.0>], 7: [<Interval t1:0 t2:6 payload:1.0>], 22: [<Interval t1:0 t2:6 payload:1.0>], 36: [<Interval t1:0 t2:6 payload:1.0>], 43: [<Interval t1:0 t2:6 payload:1.0>], 53: [<Interval t1:0 t2:6 payload:1.0>], 69: [<Interval t1:0 t2:6 payload:1.0>]}

In [88]:
def print_lfs(iset):
    print(np.array(
        iset.map(
            lambda intrvl: Interval(intrvl['bounds'], [intrvl['payload']])
        ).coalesce(
            ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
        ).get_intervals()[0]['payload']))

In [92]:
for k in gt_ism_patient.filter(payload_satisfies(lambda p: p == 1)):
    print_lfs(lf_ism[k])

[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0. -1.  1.]
 [-1.  1.  0.  0.  1.]
 [-1.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1. -1.  1.]
 [ 1.  1.  0. -1.  1.]]
[[ 0. -1.  0.  0. -1.]
 [ 1.  1.  1.  0. -1.]
 [ 1. -1.  1.  0. -1.]
 [ 1.  1.  1.  0. -1.]
 [ 1.  1.  1.  0. -1.]
 [ 1. -1.  1.  0. -1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[ 1.  1.  1.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]


In [93]:
for k in gt_ism_patient.filter(payload_satisfies(lambda p: p == -1)):
    print_lfs(lf_ism[k])

[[-1.  1.  0.  0.  1.]
 [-1.  0.  0.  0. -1.]
 [-1.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[ 0.  0.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 1.  1.  1.  0. -1.]
 [ 1.  1.  1.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1. -1.  1.]]
[[ 1.  1.  1. -1. -1.]
 [ 0.  0.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]
[[-1. -1.  0. -1. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 1. -1.  1. -1. -1.]
 [-1. -1.  0.  0. -1.]
 [ 0. -1.  0.  0.  0.]
 [ 0. -1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]]
[[-1. -1.  0.  0. -1.]
 [ 0.  0.  0.  1. -1.]
 [ 0.  1.  1.  1. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0. -1.  0.]
 [ 1.  1.  0. -1.  1.]]
[[ 0. -1.  0.  0.  0.]
 [ 0. -1.  0.  0. -1.]
 [ 0.  0.  0.  1.  1.]
 [ 0.  1.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  1.  0. -1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0